<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/03-Pandas/E2-Reading_from_Web_Pages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

# Retrieve Country Population From Wikipedia

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_population_(United_Nations)'
df_list = pd.read_html(
            url, # Specify the page 
            match='Population', # Write a pattern that is unique to the table we are interested in
            header=0 # which line of the table to use as a header
        )

len(df_list) # check how many tables match

In [ ]:
df_population = df_list[0]
df_population

In [ ]:
# Remove the footnote notations in square brackets
df_population = df_population.replace(
    to_replace = r'(.*)\[.\]',
    value = r'\1',
    regex=True
)


In [ ]:
# Rename the columns
df_population.columns = ['Country', 'Region', 'Subegion', 'Population_2018', 'Population_2019', 'Change']
# Keep only the columns we need
df_population = df_population[ ['Country', 'Population_2019'] ]

In [ ]:
df_population.dtypes

# Retrieve Life Expectancy From Wikipedia

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_life_expectancy'
df_list = pd.read_html(
            url, # Specify the page 
            match='Life expectancy at birth', # Write a pattern that is unique to the table we are interested in
            header=0 # which line of the table to use as a header
        )

len(df_list)

In [ ]:
df_who=df_list[0]
df_who

In [ ]:
# Keep only two columns
df_who = df_who.filter( items = ['Countries', 'Life expectancy at birth'] )

In [ ]:
# Delete the first two lines of the dataset
df_who = df_who.drop([0,1])

In [ ]:
# Convert the 'Life expectancy at birth' column to numeric
# and store it in a column called "Life_Expectancy"
df_who['Life_expectancy'] = pd.to_numeric(df_who['Life expectancy at birth'])

In [ ]:
# Delete the original column
df_who = df_who.drop('Life expectancy at birth', axis='columns')

In [ ]:
df_who

# Joining Tables on Country and Plotting

In [ ]:
join = pd.merge(df_population, df_who, how = 'inner', left_on= 'Country', right_on = 'Countries' )
join

In [ ]:
join.dtypes

In [ ]:
join.plot.scatter(x='Population_2019', y = 'Life_expectancy', logx=True, figsize=(10,10))